### Convolution Neural Network

Design reasoning
1. Fashion data is more complex than digits, so we use more filters
2. Choose Avg pooling on first conv layer to preserve more information
3. Use Dropout to avoid overfitting. Although current start-of-art is Dropconnect, but it is not supported by PyTorch
4. Use two layers of FC to reduce dimension

In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
import torch.optim as optim
import torchvision.datasets as dset
import numpy as np

In [12]:
from torch_data_utils import load_fashion_mnist
import mlp_torch
reload(mlp_torch)

<module 'mlp_torch' from 'mlp_torch.py'>

In [4]:
train_loader, test_loader = load_fashion_mnist()

Processing...
Done!


In [13]:
model = mlp_torch.Model(784, 500, 256, 64, 10)

In [14]:
max_epochs = 30
report_size = 1000
iter = 0
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
log = {}

for epoch in range(max_epochs):
    for i, (features, labels) in enumerate(train_loader):
        
        features = Variable(features)
        labels = Variable(labels)
        
        optimizer.zero_grad()
        outputs = model(features)
        
        loss = criterion(outputs, labels)
        loss.backward()
        
        optimizer.step()
        
        iter += 1
        
        if iter % report_size == 0:
            correct = 0.
            total = 0.
            
            for features, labels in test_loader:
                features = Variable(features)
                result = model(features)
                
                _, predicted = torch.max(result.data, 1)
                
                total += labels.size(0)
                correct += (predicted == labels).sum()
            
            accuracy = correct / total
            log[iter] = accuracy
            print('Iter: {}. Accuracy: {}'.format(iter, accuracy))

Iter: 1000. Accuracy: 0.5443
Iter: 2000. Accuracy: 0.5616
Iter: 3000. Accuracy: 0.567
Iter: 4000. Accuracy: 0.563
Iter: 5000. Accuracy: 0.5626
Iter: 6000. Accuracy: 0.5648
Iter: 7000. Accuracy: 0.5634
Iter: 8000. Accuracy: 0.5632
Iter: 9000. Accuracy: 0.5682
Iter: 10000. Accuracy: 0.5572
Iter: 11000. Accuracy: 0.5668
Iter: 12000. Accuracy: 0.5636
Iter: 13000. Accuracy: 0.564
Iter: 14000. Accuracy: 0.5682
Iter: 15000. Accuracy: 0.5675
Iter: 16000. Accuracy: 0.5637
Iter: 17000. Accuracy: 0.5694
Iter: 18000. Accuracy: 0.5638
Iter: 19000. Accuracy: 0.5664
Iter: 20000. Accuracy: 0.561
Iter: 21000. Accuracy: 0.5662
Iter: 22000. Accuracy: 0.5592
Iter: 23000. Accuracy: 0.5661
Iter: 24000. Accuracy: 0.6536
Iter: 25000. Accuracy: 0.657
Iter: 26000. Accuracy: 0.6572
Iter: 27000. Accuracy: 0.66
Iter: 28000. Accuracy: 0.6572


In [ ]:
%matplotlib inline  
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
matplotlib.rcParams.update({'font.size': 14})
figsize = (8, 5)
 
 
def plot(test_logs, size = figsize):
     
    plt.figure(1, figsize=size)
 
    lists = sorted(test_logs.items()) 
    x, y = zip(*lists) 
    plt.plot(x, y, label = 'Testing')
 
    plt.ylabel('Accuracy ')
    plt.xlabel('Number of Iterations')
    plt.title('Test Accuracy VS. Number of Iterations')

In [ ]:
plot(log)